In [4]:
pip install lightgbm


   ---------------------------------------- 0.0/1.5 MB ? eta -:--:--
   -- ------------------------------------- 0.1/1.5 MB 3.0 MB/s eta 0:00:01
   --------------- ------------------------ 0.6/1.5 MB 7.0 MB/s eta 0:00:01
   ----------------------------- ---------- 1.1/1.5 MB 8.6 MB/s eta 0:00:01
   ---------------------------------------- 1.5/1.5 MB 9.2 MB/s eta 0:00:00


In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split,KFold
from sklearn.ensemble import RandomForestClassifier,VotingClassifier,StackingClassifier
from sklearn.ensemble import AdaBoostClassifier, StackingClassifier
from lightgbm import LGBMClassifier,early_stopping
from sklearn.neural_network import MLPClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import accuracy_score,log_loss
from sklearn.experimental import enable_iterative_imputer  # noqa
from sklearn.linear_model import BayesianRidge
from sklearn.impute import IterativeImputer, SimpleImputer
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

D:\anaconda\Lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:15: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


In [4]:
!pip install clustered-imputation==1.0.2

In [6]:
df = pd.read_csv("framingham.csv")

In [8]:
df.head(3)

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0


In [14]:
df.shape

(4240, 16)

In [16]:
df.isnull().sum()

male                 0
age                  0
education          105
currentSmoker        0
cigsPerDay          29
BPMeds              53
prevalentStroke      0
prevalentHyp         0
diabetes             0
totChol             50
sysBP                0
diaBP                0
BMI                 19
heartRate            1
glucose            388
TenYearCHD           0
dtype: int64

In [30]:
X,y = df.drop(['TenYearCHD'], axis=1), df.TenYearCHD

In [32]:
X

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4235,0,48,2.0,1,20.0,NaN,0,0,0,248.0,131.0,72.0,22.00,84.0,86.0
4236,0,44,1.0,1,15.0,0.0,0,0,0,210.0,126.5,87.0,19.16,86.0,NaN
4237,0,52,2.0,0,0.0,0.0,0,0,0,269.0,133.5,83.0,21.47,80.0,107.0
4238,1,40,3.0,0,0.0,0.0,0,1,0,185.0,141.0,98.0,25.60,67.0,72.0


In [35]:
def em_train(df, n_components=2, max_iter=10):
    # Separate numerical and categorical columns
    copy_df = df.copy()
    num_cols = copy_df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = copy_df.select_dtypes(include=['object', 'category']).columns.tolist()
    
    # Initialize GMM for numerical columns
    gmm_imputer = GaussianMixture(n_components=n_components, max_iter=max_iter)
    copy_df[num_cols] = em_impute(gmm_imputer, copy_df[num_cols], max_iter)
    
    # Store mode for categorical columns for test set imputation
    cat_modes = {col: copy_df[col].mode()[0] for col in cat_cols}
    
    return copy_df, gmm_imputer, cat_modes

def em_impute(gmm, data, max_iter):
    data_imputed = data.copy()
    
    for i in range(max_iter):
        # Fit GMM on observed data (ignoring rows with missing values in numerical columns)
        gmm.fit(data_imputed.dropna())
        
        # Impute missing values using the fitted GMM
        missing_mask = data.isna()
        for col in data.columns:
            col_missing = missing_mask[col]
            if col_missing.any():  # If there are missing values in this column
                # Generate new samples for missing values in the column
                samples = gmm.sample(col_missing.sum())[0]
                data_imputed.loc[col_missing, col] = samples[:, 0]  # Use only the first component for the column
                
    return data_imputed


def mice_train(df, max_iter=10):
    # Separate numerical and categorical columns
    copy_df = df.copy()
    num_cols = copy_df.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = copy_df.select_dtypes(include=['object', 'category']).columns.tolist()

    mice_imputer = IterativeImputer(max_iter=max_iter, random_state=0)
    copy_df[num_cols] = mice_imputer.fit_transform(copy_df[num_cols])
    return copy_df, mice_imputer

In [37]:
from clustered_imputation import clusterImputer
clustered_mice_X,clustered_em_X = X.copy(),X.copy()
mice_X, mice_imputer = mice_train(X)
em_X, em_imputer,_ = em_train(X, n_components=2, max_iter=10)
clustered_mice = clusterImputer(clustered_mice_X , "mice" ,"mean" , 0.4 ,10)
clustered_em = clusterImputer(clustered_em_X , "em" ,"mean" , 0.4 ,10)
clustered_mice.impute()
clustered_em.impute()

In [39]:
clustered_em_X.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
dtype: int64

In [41]:
clustered_mice_X.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
dtype: int64

In [43]:
em_X.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
dtype: int64

In [45]:
mice_X.isnull().sum()

male               0
age                0
education          0
currentSmoker      0
cigsPerDay         0
BPMeds             0
prevalentStroke    0
prevalentHyp       0
diabetes           0
totChol            0
sysBP              0
diaBP              0
BMI                0
heartRate          0
glucose            0
dtype: int64

In [47]:
def build_kfold_model(X, y, X_test, ModelClass, params):
    kf = KFold(n_splits=5, random_state=0, shuffle=True)
    models = []
    X_test_preds = np.zeros(X_test.shape[0])  # Initialize with zeros
    X_train_loss = 0
    X_val_loss = 0
    y = np.array(y)
    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]  # Access y as a NumPy array

        model = ModelClass(**params)
        model.fit(X_train, y_train)

        train_preds = model.predict(X_train)
        val_preds = model.predict(X_val)
        test_preds = model.predict(X_test)

        train_loss = accuracy_score(y_train, train_preds)
        val_loss = accuracy_score(y_val, val_preds)
        X_train_loss += train_loss
        X_val_loss += val_loss
        X_test_preds += test_preds

        models.append(model)

    # Average predictions for test set
    X_test_preds /= kf.n_splits
    X_test_preds = np.where(X_test_preds < 0.5, 0, 1)  # Convert to binary predictions

    return models, X_test_preds, X_train_loss / kf.n_splits, X_val_loss / kf.n_splits

In [49]:
def get_model_kfold_results_am():
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':  {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':{'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X = x_train_clustered_mice
    y = y_train_clustered_mice
    new_test_df = x_test_clustered_mice
    for i in models:
        trained_models,X_test_preds,X_train_score,X_val_score = build_kfold_model(X,y,new_test_df,models[i]['model'],models[i]['params'])
        models[i]['trained_models'] = trained_models
        models[i]['test_preds'] = X_test_preds
        print(f"Accuracy Score For mode {i} is --- Train accuracy {X_train_score} and --- Val accuracy {X_val_score}")

    return models


def get_model_kfold_results_em():
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params':  {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params':{'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X = x_train_clustered_em
    y = y_train_clustered_em
    new_test_df = np.array(x_test_clustered_em)
    for i in models:
        trained_models,X_test_preds,X_train_score,X_val_score = build_kfold_model(X,y,new_test_df,models[i]['model'],models[i]['params'])
        models[i]['trained_models'] = trained_models
        models[i]['test_preds'] = X_test_preds
        print(f"Accuracy Score For mode {i} is --- Train accuracy {X_train_score} and --- Val accuracy {X_val_score}")

    return models



def get_model_kfold_results_plain_mice():
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':  {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params':{'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params': {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X = x_train_plain_mice
    y = y_train_plain_mice
    new_test_df = np.array(x_test_plain_mice)
    for i in models:
        trained_models,X_test_preds,X_train_score,X_val_score = build_kfold_model(X,y,new_test_df,models[i]['model'],models[i]['params'])
        models[i]['trained_models'] = trained_models
        models[i]['test_preds'] = X_test_preds
        print(f"Accuracy Score For mode {i} is --- Train accuracy {X_train_score} and --- Val accuracy {X_val_score}")

    return models



def get_model_kfold_results_plain_em():
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X = x_train_plain_em
    y = y_train_plain_em
    new_test_df = np.array(x_test_plain_em)
    for i in models:
        trained_models,X_test_preds,X_train_score,X_val_score = build_kfold_model(X,y,new_test_df,models[i]['model'],models[i]['params'])
        models[i]['trained_models'] = trained_models
        models[i]['test_preds'] = X_test_preds
        print(f"Accuracy Score For mode {i} is --- Train accuracy {X_train_score} and --- Val accuracy {X_val_score}")

    return models

In [51]:
def predict_am(X_train, y_train, X_test, y_test):
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    
    for model_name, model_dict in models.items():
        model = model_dict['model'](**model_dict['params'])  # Instantiate model
        model.fit(X_train, y_train)  # Train the model
        
        # Make predictions
        preds = model.predict(X_test)
        
        # Calculate accuracy score
        accuracy = accuracy_score(y_test, preds)
        print(f"{model_name} Model Accuracy: {accuracy:.4f}")


def predict_em(X_train, y_train, X_test, y_test):
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    for model_name, model_dict in models.items():
        model = model_dict['model'](**model_dict['params'])  # Instantiate model
        model.fit(X_train, y_train)  # Train the model
        
        # Make predictions
        preds = model.predict(X_test)
        
        # Calculate accuracy score
        accuracy = accuracy_score(y_test, preds)
        print(f"{model_name} Model Accuracy: {accuracy:.4f}")



def predict_plain_mice(X_train, y_train, X_test, y_test):
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }

    X_test = np.array(X_test)
    y_test = np.array(y_test)

    for model_name, model_dict in models.items():
        model = model_dict['model'](**model_dict['params'])  # Instantiate model
        model.fit(X_train, y_train)  # Train the model
        
        # Make predictions
        preds = model.predict(X_test)
        
        # Calculate accuracy score
        accuracy = accuracy_score(y_test, preds)
        print(f"{model_name} Model Accuracy: {accuracy:.4f}")


def predict_plain_em(X_train, y_train, X_test, y_test):
    models = {
        'XGB': {
        'model': XGBClassifier,
        'params':   {'n_estimators': 1000, 'max_depth': 8, 'alpha': 1.9738594118973054, 'lambda': 4.009774721904594, 'learning_rate': 0.06723886268535728, 'subsample': 0.8046187868546699, 'colsample_bytree': 0.6422765532436576},
        'trained_models': [],
        'test_preds' : []
    },
       'LGBM': {
        'model': LGBMClassifier,
        'params': {'lambda': 0.012092546196007962, 'alpha': 31.687502527930103, 'num_leaves': 91, 'n_estimators': 1000, 'max_depth': 6, 'learning_rate': 0.3393288299428538, 'bagging_fraction': 0.9195007323386031, 'mi_split_gain': 0.03928856728583821, 'lambda_l1': 0.10440536035552132, 'lambda_l2': 0.19185454581480765,'verbose':-100},
        'trained_models': [],   
        'test_preds' : []
    },    
    'HIST' :{
        'model' : HistGradientBoostingClassifier,
        'params':  {'learning_rate': 2.4832513288186244, 'max_leaf_nodes': 31, 'l2_regularization': 8.930490898699928, 'max_depth': 10, 'min_samples_leaf': 50},
        'trained_models': [],
        'test_preds' : []
    },
        'RF': {
        'model' : RandomForestClassifier,
        'params': {'n_estimators': 1000, 'max_depth': 9, 'class_weight': {0: 1, 1: 5}, 'min_samples_split': 6},
        'trained_models': [],
        'test_preds' : []
        }
  }
    X_test = np.array(X_test)
    y_test = np.array(y_test)

    for model_name, model_dict in models.items():
        model = model_dict['model'](**model_dict['params'])  # Instantiate model
        model.fit(X_train, y_train)  # Train the model
        
        # Make predictions
        preds = model.predict(X_test)
        
        # Calculate accuracy score
        accuracy = accuracy_score(y_test, preds)
        print(f"{model_name} Model Accuracy: {accuracy:.4f}")


In [53]:
x_train_plain_mice,x_test_plain_mice,y_train_plain_mice,y_test_plain_mice = train_test_split(mice_X,y,random_state=42,stratify=y,test_size = 0.3)
x_train_plain_em,x_test_plain_em,y_train_plain_em,y_test_plain_em = train_test_split(em_X,y,random_state=42,stratify=y,test_size = 0.3)
x_train_clustered_mice,x_test_clustered_mice,y_train_clustered_mice,y_test_clustered_mice = train_test_split(clustered_mice_X,y,random_state=42,stratify=y,test_size = 0.3)
x_train_clustered_em,x_test_clustered_em,y_train_clustered_em,y_test_clustered_em = train_test_split(clustered_em_X,y,random_state=42,stratify=y,test_size = 0.3)

In [55]:
scaler_corr_mice = StandardScaler()
scaler_corr_em = StandardScaler()
scaler_plain_mice = StandardScaler()
scaler_plain_em = StandardScaler()

# Apply StandardScaler on correlation-based MICE imputed data
x_train_clustered_mice = scaler_corr_mice.fit_transform(x_train_clustered_mice)
x_test_clustered_mice = scaler_corr_mice.transform(x_test_clustered_mice)

# Apply StandardScaler on correlation-based EM imputed data
x_train_clustered_em = scaler_corr_em.fit_transform(x_train_clustered_em)
x_test_clustered_em = scaler_corr_em.transform(x_test_clustered_em)

# Apply StandardScaler on plain MICE imputed data
x_train_plain_mice = scaler_plain_mice.fit_transform(x_train_plain_mice)
x_test_plain_mice = scaler_plain_mice.transform(x_test_plain_mice)

# Apply StandardScaler on plain EM imputed data
x_train_plain_em = scaler_plain_em.fit_transform(x_train_plain_em)
x_test_plain_em = scaler_plain_em.transform(x_test_plain_em)

In [57]:
results_am = get_model_kfold_results_am()

Accuracy Score For mode XGB is --- Train accuracy 0.9983995743360085 and --- Val accuracy 0.8355851942698486
Accuracy Score For mode LGBM is --- Train accuracy 1.0 and --- Val accuracy 0.8298578817971733
Accuracy Score For mode HIST is --- Train accuracy 0.9882055602358889 and --- Val accuracy 0.7900948779532253
Accuracy Score For mode RF is --- Train accuracy 0.9438177803396444 and --- Val accuracy 0.8140187711857187


In [58]:
predict_am(x_train_clustered_mice,y_train_clustered_mice  ,x_test_clustered_mice , y_test_clustered_mice)

XGB Model Accuracy: 0.8325
LGBM Model Accuracy: 0.8192
HIST Model Accuracy: 0.7987
RF Model Accuracy: 0.7964


In [61]:
results_em = get_model_kfold_results_em()

Accuracy Score For mode XGB is --- Train accuracy 0.9983995743360085 and --- Val accuracy 0.8355851942698486
Accuracy Score For mode LGBM is --- Train accuracy 1.0 and --- Val accuracy 0.8298578817971733
Accuracy Score For mode HIST is --- Train accuracy 0.9882055602358889 and --- Val accuracy 0.7900948779532253
Accuracy Score For mode RF is --- Train accuracy 0.944912552653749 and --- Val accuracy 0.8146927396505811


In [62]:
predict_plain_mice(x_train_plain_mice,y_train_plain_mice  ,x_test_plain_mice , y_test_plain_mice)

XGB Model Accuracy: 0.8310
LGBM Model Accuracy: 0.8263
HIST Model Accuracy: 0.7728
RF Model Accuracy: 0.7925


In [65]:
results_em_plain = get_model_kfold_results_plain_em()

Accuracy Score For mode XGB is --- Train accuracy 0.9990734713785306 and --- Val accuracy 0.838953333219775
Accuracy Score For mode LGBM is --- Train accuracy 1.0 and --- Val accuracy 0.8349106580135247
Accuracy Score For mode HIST is --- Train accuracy 0.9574706690905866 and --- Val accuracy 0.7877260519756304
Accuracy Score For mode RF is --- Train accuracy 0.9377524941249501 and --- Val accuracy 0.8099795027282379


In [67]:
predict_plain_em(x_train_plain_em,y_train_plain_em  ,x_test_plain_em , y_test_plain_em)

XGB Model Accuracy: 0.8294
LGBM Model Accuracy: 0.8255
HIST Model Accuracy: 0.7909
RF Model Accuracy: 0.7940
